In [1]:
import pandas as pd

In [2]:
# products_with_categories.csv
path = "https://raw.githubusercontent.com/Tehnik762/Eniac_discounts/main/products_with_categories.csv"
products_df = pd.read_csv(path)
products_df.drop(columns=["Unnamed: 0"])

# orders_qu.csv
path = "https://raw.githubusercontent.com/Tehnik762/Eniac_discounts/main/orders_qu.csv"
orders_df = pd.read_csv(path)

# orderlines_df.csv
path = "https://raw.githubusercontent.com/Tehnik762/Eniac_discounts/main/orderlines_qu.csv"
orderlines_df = pd.read_csv(path)

# brands_cl.csv
path = "https://raw.githubusercontent.com/Tehnik762/Eniac_discounts/main/brands.csv"
brands = pd.read_csv(path)

## Preparing Discounts_DF

In [21]:
discounts_df = orderlines_df.merge(products_df, how="left", on="sku")[["sku", "unit_price", "price", "date", "id", "id_order" ]]

In [23]:
discounts_df["discounts"] = discounts_df["price"] - discounts_df["unit_price"]

In [15]:
discounts_df["discounts"].describe()

count    82156.000000
mean        24.928848
std         45.363207
min       -785.050000
25%          5.000000
50%         14.720000
75%         30.000000
max       1320.000000
Name: discounts, dtype: float64

### Removing the outliers

In [28]:
q1 = discounts_df["discounts"].quantile(0.25)
q3 = discounts_df["discounts"].quantile(0.75)
iqr = q3-q1
discounts_df.shape

(82946, 7)

In [32]:
discounts_df = discounts_df.loc[(discounts_df["discounts"] > (q1 - 1.5*iqr)) & (discounts_df["discounts"] < (q3 + 1.5*iqr))]
discounts_df

,sku,unit_price,price,date,id,id_order,discounts
0,OWC0100,47.49,60.99,2017-01-01 01:46:16,1119116,299545,13.50
1,IOT0014,18.99,22.95,2017-01-01 01:50:34,1119119,299546,3.96
2,APP0700,72.19,89.00,2017-01-01 01:54:11,1119120,295347,16.81
3,BEA0065,256.49,299.95,2017-01-01 02:18:45,1119133,299555,43.46
4,CRU0039-A,60.90,76.99,2017-01-01 02:20:14,1119134,299556,16.09
...,...,...,...,...,...,...,...
82940,OWC0035-2,71.89,87.98,2018-03-14 13:52:18,1650186,525853,16.09
82941,APP1922,161.00,179.00,2018-03-14 13:55:15,1650193,527394,18.00
82943,LAC0227,249.99,299.99,2018-03-14 13:56:02,1650196,527384,50.00
82944,JBL0122,42.99,49.95,2018-03-14 13:56:38,1650198,527397,6.96


In [34]:
orders_df = orders_df.loc[orders_df.order_id.isin(discounts_df['id_order'])]

In [38]:
orderlines_df = orderlines_df.loc[orderlines_df.id_order.isin(discounts_df['id_order'])]

Now we are ready to work with discounts!